# Ollama PDF RAG Notebook

## Import Libraries


In [3]:
import time 

inicial_time = time.time()

time.sleep(5)
final_time = time.time()

time_spent = final_time - inicial_time

print(time_spent)

5.00043797492981


In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [ ]:
"""
Streamlit application for PDF-based Retrieval-Augmented Generation (RAG) using Ollama + LangChain.

This application allows users to upload a PDF, process it,
and then ask questions about the content using a selected language model.
"""

import streamlit as st
import logging
import os
import tempfile
import shutil
import pdfplumber
import ollama

from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from typing import List, Tuple, Dict, Any, Optional

# Set protobuf environment variable to avoid error messages
# This might cause some issues with latency but it's a tradeoff
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Streamlit page configuration
st.set_page_config(
    page_title="Ollama PDF RAG Streamlit UI",
    page_icon="🎈",
    layout="wide",
    initial_sidebar_state="collapsed",
)

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logger = logging.getLogger(__name__)


@st.cache_resource(show_spinner=True)
def extract_model_names(
    models_info: Dict[str, List[Dict[str, Any]]],
) -> Tuple[str, ...]:
    """
    Extract model names from the provided models information.

    Args:
        models_info (Dict[str, List[Dict[str, Any]]]): Dictionary containing information about available models.

    Returns:
        Tuple[str, ...]: A tuple of model names.
    """
    logger.info("Extracting model names from models_info")
    models_data = [{"name": model.model} for model in models_info.models]
    model_names = tuple(model["name"] for model in models_data)
    logger.info(f"Extracted model names: {model_names}")
    return model_names


def create_vector_db(file_upload) -> Chroma:
    """
    Create a vector database from an uploaded PDF file.

    Args:
        file_upload (st.UploadedFile): Streamlit file upload object containing the PDF.

    Returns:
        Chroma: A vector store containing the processed document chunks.
    """
    logger.info(f"Creating vector DB from file upload: {file_upload.name}")
    temp_dir = tempfile.mkdtemp()

    path = os.path.join(temp_dir, file_upload.name)
    with open(path, "wb") as f:
        f.write(file_upload.getvalue())
        logger.info(f"File saved to temporary path: {path}")
        loader = UnstructuredPDFLoader(path)
        data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
    chunks = text_splitter.split_documents(data)
    logger.info("Document split into chunks")

    # Updated embeddings configuration
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    vector_db = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        collection_name="myRAG"
    )
    logger.info("Vector DB created")

    shutil.rmtree(temp_dir)
    logger.info(f"Temporary directory {temp_dir} removed")
    return vector_db


def process_question(question: str, vector_db: Chroma, selected_model: str) -> str:
    """
    Process a user question using the vector database and selected language model.

    Args:
        question (str): The user's question.
        vector_db (Chroma): The vector database containing document embeddings.
        selected_model (str): The name of the selected language model.

    Returns:
        str: The generated response to the user's question.
    """
    logger.info(f"Processing question: {question} using model: {selected_model}")
    
    # Initialize LLM
    llm = ChatOllama(model=selected_model)
    
    # Query prompt template
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI language model assistant. Your task is to generate 2
        different versions of the given user question to retrieve relevant documents from
        a vector database. By generating multiple perspectives on the user question, your
        goal is to help the user overcome some of the limitations of the distance-based
        similarity search. Provide these alternative questions separated by newlines.
        Original question: {question}""",
    )

    # Set up retriever
    retriever = MultiQueryRetriever.from_llm(
        vector_db.as_retriever(), 
        llm,
        prompt=QUERY_PROMPT
    )

    # RAG prompt template
    template = """Answer the question based ONLY on the following context:
    {context}
    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    # Create chain
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    response = chain.invoke(question)
    logger.info("Question processed and response generated")
    return response


@st.cache_data
def extract_all_pages_as_images(file_upload) -> List[Any]:
    """
    Extract all pages from a PDF file as images.

    Args:
        file_upload (st.UploadedFile): Streamlit file upload object containing the PDF.

    Returns:
        List[Any]: A list of image objects representing each page of the PDF.
    """
    logger.info(f"Extracting all pages as images from file: {file_upload.name}")
    pdf_pages = []
    with pdfplumber.open(file_upload) as pdf:
        pdf_pages = [page.to_image().original for page in pdf.pages]
    logger.info("PDF pages extracted as images")
    return pdf_pages


def delete_vector_db(vector_db: Optional[Chroma]) -> None:
    """
    Delete the vector database and clear related session state.

    Args:
        vector_db (Optional[Chroma]): The vector database to be deleted.
    """
    logger.info("Deleting vector DB")
    if vector_db is not None:
        vector_db.delete_collection()
        st.session_state.pop("pdf_pages", None)
        st.session_state.pop("file_upload", None)
        st.session_state.pop("vector_db", None)
        st.success("Collection and temporary files deleted successfully.")
        logger.info("Vector DB and related session state cleared")
        st.rerun()
    else:
        st.error("No vector database found to delete.")
        logger.warning("Attempted to delete vector DB, but none was found")


def main() -> None:
    """
    Main function to run the Streamlit application.
    """
    st.subheader("🧠 Ollama PDF RAG playground", divider="gray", anchor=False)

    # Get available models
    models_info = ollama.list()
    available_models = extract_model_names(models_info)

    # Create layout
    col1, col2 = st.columns([1.5, 2])

    # Initialize session state
    if "messages" not in st.session_state:
        st.session_state["messages"] = []
    if "vector_db" not in st.session_state:
        st.session_state["vector_db"] = None
    if "use_sample" not in st.session_state:
        st.session_state["use_sample"] = False

    # Model selection
    if available_models:
        selected_model = col2.selectbox(
            "Pick a model available locally on your system ↓", 
            available_models,
            key="model_select"
        )

    # Add checkbox for sample PDF
    use_sample = col1.toggle(
        "Use sample PDF (Scammer Agent Paper)", 
        key="sample_checkbox"
    )
    
    # Clear vector DB if switching between sample and upload
    if use_sample != st.session_state.get("use_sample"):
        if st.session_state["vector_db"] is not None:
            st.session_state["vector_db"].delete_collection()
            st.session_state["vector_db"] = None
            st.session_state["pdf_pages"] = None
        st.session_state["use_sample"] = use_sample

    if use_sample:
        # Use the sample PDF
        sample_path = "scammer-agent.pdf"
        if os.path.exists(sample_path):
            if st.session_state["vector_db"] is None:
                with st.spinner("Processing sample PDF..."):
                    loader = UnstructuredPDFLoader(file_path=sample_path)
                    data = loader.load()
                    text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
                    chunks = text_splitter.split_documents(data)
                    st.session_state["vector_db"] = Chroma.from_documents(
                        documents=chunks,
                        embedding=OllamaEmbeddings(model="nomic-embed-text"),
                        collection_name="myRAG"
                    )
                    # Open and display the sample PDF
                    with pdfplumber.open(sample_path) as pdf:
                        pdf_pages = [page.to_image().original for page in pdf.pages]
                        st.session_state["pdf_pages"] = pdf_pages
        else:
            st.error("Sample PDF file not found in the current directory.")
    else:
        # Regular file upload with unique key
        file_upload = col1.file_uploader(
            "Selecione um arquivo PDF ↓", 
            type="pdf", 
            accept_multiple_files=False,
            key="pdf_uploader"
        )

        if file_upload:
            if st.session_state["vector_db"] is None:
                with st.spinner("Processando upload do PDF..."):
                    st.session_state["vector_db"] = create_vector_db(file_upload)
                    pdf_pages = extract_all_pages_as_images(file_upload)
                    st.session_state["pdf_pages"] = pdf_pages

    # Display PDF if pages are available
    if "pdf_pages" in st.session_state and st.session_state["pdf_pages"]:
        # PDF display controls
        zoom_level = col1.slider(
            "Zoom Level", 
            min_value=100, 
            max_value=1000, 
            value=700, 
            step=50,
            key="zoom_slider"
        )

        # Display PDF pages
        with col1:
            with st.container(height=410, border=True):
                # Removed the key parameter from st.image()
                for page_image in st.session_state["pdf_pages"]:
                    st.image(page_image, width=zoom_level)

    # Delete collection button
    delete_collection = col1.button(
        "⚠️ Deletar collection", 
        type="secondary",
        key="delete_button"
    )

    if delete_collection:
        delete_vector_db(st.session_state["vector_db"])

    # Chat interface
    with col2:
        message_container = st.container(height=500, border=True)

        # Display chat history
        for i, message in enumerate(st.session_state["messages"]):
            avatar = "🤖" if message["role"] == "assistant" else "😎"
            with message_container.chat_message(message["role"], avatar=avatar):
                st.markdown(message["content"])

        # Chat input and processing
        if prompt := st.chat_input("Enter a prompt here...", key="chat_input"):
            try:
                # Add user message to chat
                st.session_state["messages"].append({"role": "user", "content": prompt})
                with message_container.chat_message("user", avatar="😎"):
                    st.markdown(prompt)

                # Process and display assistant response
                with message_container.chat_message("assistant", avatar="🤖"):
                    with st.spinner(":green[processing...]"):
                        if st.session_state["vector_db"] is not None:
                            response = process_question(
                                prompt, st.session_state["vector_db"], selected_model
                            )
                            st.markdown(response)
                        else:
                            st.warning("Please upload a PDF file first.")

                # Add assistant response to chat history
                if st.session_state["vector_db"] is not None:
                    st.session_state["messages"].append(
                        {"role": "assistant", "content": response}
                    )

            except Exception as e:
                st.error(e, icon="⛔️")
                logger.error(f"Error processing prompt: {e}")
        else:
            if st.session_state["vector_db"] is None:
                st.warning("Faça upload de um arquivo PDF ou selecione o exemplo para começar o chat...")


if __name__ == "__main__":
    main()

2024-12-05 22:57:24.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 22:57:24.338 No runtime found, using MemoryCacheStorageManager
2024-12-05 22:57:24.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 22:57:24.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 22:57:24 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
2024-12-05 22:57:24.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 22:57:24.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 22:57:24.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 22:57:24.351 Thread 'MainThread': missing ScriptRunContext! Th

UnhashableParamError: Cannot hash argument 'models_info' (of type `ollama._types.ListResponse`) in 'extract_model_names'.

To address this, you can tell Streamlit not to hash this argument by adding a
leading underscore to the argument's name in the function signature:

```
@st.cache_resource
def extract_model_names(_models_info, ...):
    ...
```
            

In [16]:
models_info

ListResponse(models=[Model(model='mxbai-embed-large:latest', modified_at=datetime.datetime(2024, 12, 4, 23, 32, 36, 632141, tzinfo=TzInfo(-03:00)), digest='468836162de7f81e041c43663fedbbba921dcea9b9fefea135685a39b2d83dd8', size=669615493, details=ModelDetails(parent_model='', format='gguf', family='bert', families=['bert'], parameter_size='334M', quantization_level='F16')), Model(model='llava:7b', modified_at=datetime.datetime(2024, 12, 4, 22, 18, 59, 109732, tzinfo=TzInfo(-03:00)), digest='8dd30f6b0cb19f555f2c7a7ebda861449ea2cc76bf1f44e262931f45fc81d081', size=4733363377, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama', 'clip'], parameter_size='7B', quantization_level='Q4_0')), Model(model='llava:13b', modified_at=datetime.datetime(2024, 12, 4, 21, 44, 42, 194135, tzinfo=TzInfo(-03:00)), digest='0d0eb4d7f485d7d0a21fd9b0c1d5b04da481d2150a097e81b64acb59758fdef6', size=8011256494, details=ModelDetails(parent_model='', format='gguf', family='llama', 

2024-12-05 23:05:49 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
2024-12-05 23:05:49.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 23:05:49.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 23:05:49.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 23:05:49 - INFO - Extracting model names from models_info
2024-12-05 23:05:49 - INFO - Extracted model names: ('mxbai-embed-large:latest', 'llava:7b', 'llava:13b', 'nomic-embed-text:latest', 'llama3.2:latest')
2024-12-05 23:05:49.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-05 23:05:49.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


## Load PDF

In [2]:
# Load PDF
local_path = "attention.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: attention.pdf


## Split text into chunks

In [3]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 51 chunks


## Create vector database

In [5]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [6]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [7]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template = """Você é um assistente de modelo de linguagem AI. Sua tarefa é gerar 2
    versões diferentes da pergunta do usuário para recuperar documentos relevantes de
    um banco de dados vetorial. Ao gerar múltiplas perspectivas sobre a pergunta do usuário,
    seu objetivo é ajudar o usuário a superar algumas das limitações da busca por similaridade baseada em distância. 
    Forneça essas perguntas alternativas separadas por novas linhas.
    Pergunta original: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [8]:
# RAG prompt template
template = """Responda a pergunta baseado APENAS no contexto a seguir::
{context}
Pergunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [9]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [10]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [12]:
chat_with_pdf("O que é atenção segundo meu documento?")

De acordo com o seu documento, a atenção (ou attention) é uma função matemática utilizada em modelos de processamento de linguagem e aprendizado de máquina. Ela permite que um modelo "olhe" para diferentes partes de um texto ou sequência e dê mais peso àquela que é mais relevante para o tarefa específica. Isso é feito através de uma operação matemática chamada atenção escalada ao produto ponto (Scaled Dot-Product Attention).

In [13]:
# Example 1
chat_with_pdf("Qual é a ideia principal desse documento?")

A ideia principal deste documento parece ser relacionada à implementação e avaliação de modelos de processamento do linguagem baseados na atenção, especificamente o Transformer, que foi proposto por Vaswani et al. (2017) e desenvolvido pela equipe liderada por Ashish, Jakob, Noam, Llion, Lukasz e Aidan. O documento apresenta resultados de experimentação com vários modelos variados, incluindo a avaliação da eficácia do Transformer em tarefas de processamento do linguagem, como o tradução e o questionamento semântico.

Além disso, o documento também aborda questões relacionadas à compreensão da atenção em diferentes níveis de representação da linguagem, como a atenção no nível de palavras (word-level), frases (sentence-level) e documentos (document-level).

No entanto, sem mais informações específicas sobre a estrutura do documento e o contexto em que foi usado, não é possível fornecer uma resposta mais detalhada.

In [14]:
# Example 2
chat_with_pdf("De onde surgiu o conceito de atenção? E como é aplicado para melhorar a performace dos modelos?")

O conceito de atenção em redes neurais foi desenvolvido pela primeira vez por Sepp Hochreiter e Jürgen Schmidhuber em 2001, quando apresentaram o papel do gradient flow na aprendizagem de redes neuronais recorrentes. No entanto, o conceito moderno de atenção foi introduzido pelo pesquisador japonês Mikoto Okunawa em 1997, e mais tarde desenvolvido pela equipe liderada por Yoshua Bengio.

A atenção é um método utilizado para reorganizar a forma como as redes neuronais processam informações. Ela permite às redes focar melhor nas partes do input que são mais relevantes para o processo de aprendizado, em vez de processar todas as informações de uma só vez.

No contexto da aprendizagem de máquinas, a atenção é aplicada para melhorar a performance dos modelos de seqüência, como língua natural processamento e tradução. Ela funciona ao permitir às redes neuronais focar melhor nas partes do input que são mais relevantes para o processo de aprendizado.

A técnica foi inicialmente utilizada em redes neuronais recorrentes (RNNs) e mais tarde se generalizou para outras tipos de redes neuronais, incluindo redes neuronais convolucionais (CNNs).

O conceito de atenção é aplicado de várias maneiras para melhorar a performance dos modelos:

*   **Atenção de peso**: Em vez de processar todas as informações do input de uma só vez, a atenção de peso permite às redes focar melhor nas partes mais relevantes.
*   **Atenção spatial**: Além da atenção de peso, a atenção spatial permite às redes focar melhor em regiões específicas do input.
*   **Atenção temporal**: A atenção temporal permite às redes focar melhor em momentos específicos do input.

A atenção é uma técnica poderosa que tem sido amplamente utilizada em muitas aplicações da aprendizagem de máquina, incluindo processamento de língua natural, tradução e visão por computador.

## Clean up (optional)

In [27]:
# Optional: Clean up when done 
vector_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
